In [ ]:
import pandas as pd
import numpy as np
import math

from tqdm import tqdm

from dataprep.eda import create_report

from string_grouper import match_strings
from thefuzz import fuzz, process

In [ ]:
# Import data
userRatings = pd.read_excel("user_ratings.xlsx")
games = pd.read_excel("games.xlsx")
#userRatings_csv = pd.read_csv("user_ratings.csv")

In [ ]:
# Turn ratings into a categorical column
userRatings["BGGId"] = userRatings["BGGId"].astype('string')
#userRatings_csv["BGGId"] = userRatings_csv["BGGId"].astype('string')
games["BGGId"] = games["BGGId"].astype('string')

In [ ]:
# check if all games IDS are valid
assert set(userRatings["BGGId"]) <= set(games["BGGId"])

In [ ]:
# Unusuall ratings found
userRatings.describe()

In [ ]:
userRatings.info()

In [ ]:
userRatings[userRatings.Rating > 10] # 143 ratings larger than 10 most likely errors

#### Which usernames and games give unusual ratings?

In [ ]:
#
outlierRatings = pd.concat([userRatings[userRatings.Rating > 10].groupby(["Username"])["BGGId"].apply(list),#count()[["Rating"]]
userRatings[userRatings.Rating > 10].groupby(["Username"])["Rating"].apply(list).reset_index(name='outlier_ratings').set_index("Username"),
userRatings[userRatings.Rating > 10].groupby(["Username"]).count()["Rating"].reset_index(name='outlier_ratings_count').set_index("Username")], axis=1)

outlierRatingsTable = outlierRatings.merge(userRatings.groupby(["Username"]).count()[["Rating"]],on="Username",how="left").rename(columns={"Rating":"Total_ratings_count","BGGId":"outlier_BGGId"})


In [ ]:
#outlierRatingsTable.to_csv("Userrating outliers.csv")

In [ ]:
outlierRatingsTable = outlierRatingsTable.sort_values(by=["outlier_ratings_count","Total_ratings_count"], ascending=False)

In [ ]:
outlierRatingsTable

In [ ]:
pd.concat([userRatings[userRatings.Rating > 10].groupby(["BGGId"])["Username"].apply(list),#count()[["Rating"]]
userRatings[userRatings.Rating > 10].groupby(["BGGId"])["Rating"].apply(list),
userRatings[userRatings.Rating > 10].groupby(["BGGId"]).count()["Rating"]], axis=1)

In [ ]:
userRatings[userRatings.Rating < 0].shape # no negative ratings

In [ ]:
#games.merge(userRatings, on="BGGId", how="right")

In [ ]:
# remove invalid ratings
userRatings = userRatings[userRatings.Rating <= 10]

#### Use of the standard  EDA packages to get some insight on the ratings variable

In [ ]:
create_report(userRatings)

### Username similarity

Let's see how many are extremely similar

In [ ]:
# Using the match strings functionality to select  similar usernames
matches = match_strings(pd.Series(userRatings["Username"].unique()),min_similarity = 0.7)
#matches[matches['left_side'] != matches['right_side']].head()


In [ ]:
def length_Cal(data):
    """
        Since match_strings returns a table with duplicate similarities 
        ( alex W and alexW will be one row and again alexW and alexW will be another column)
        right now we are comparing the length of the usernames and returing the longer one to a list which will be made into a column
    """
    lenlist = []
    for i,r in data.iterrows():
        lenlist.append(max([r.left_side,r.right_side],key=len))
    return lenlist

In [ ]:
# Remove similarity measure of a username with itself. Since it will be identical to itself
new_matches = matches[matches['left_side'] != matches['right_side']]

# Remove rows that are duplicates of earch other (both rows show similarity between the same two usernames but in different order)
new_matches["duplicates"] = length_Cal(new_matches)

In [ ]:
cosine_similarity = new_matches.drop_duplicates(subset="duplicates").sort_values(by="similarity", ascending=False).drop(columns=["left_index","right_index","duplicates"])


In [ ]:
cosine_similarity

In [ ]:
# Create a dataframe with games grouped by username
rating_games = userRatings.groupby(["Username"])["BGGId"].apply(list)
rating_games = rating_games.reset_index()
rating_games

In [ ]:
# Create a dataframe with ratings grouped by username

rating_list = userRatings.groupby(["Username"])["Rating"].apply(list).reset_index()

ratings_count = userRatings.groupby("Username").count()[["Rating"]].reset_index()
ratings_count

#### Compile information on similar usernames 
left_side: username
right_side: username

right_rating_count: total number of ratings by user named in right_side
left_rating_count: total number of ratings by user named in left_side

right_ratings: list of all ratings by user named in right_side
left_ratings: list of all ratings by user named in left_side

right_BGGId: list of all games rated by user named in right_side
left_BGGId: list of all games rated by user named in left_side

common_games_count: number of games rated by both users named in left_side and right_side 


In [ ]:
cosine_similarity = cosine_similarity.merge(ratings_count, right_on="Username",left_on="right_side",how="left").drop(columns="Username").rename(columns={"Rating":"right_rating_count"})


In [ ]:
cosine_similarity = ratings_count.merge(cosine_similarity, right_on="left_side",left_on="Username",how="right").drop(columns="Username").rename(columns={"Rating":"left_rating_count"})


In [ ]:
cosine_similarity = cosine_similarity.merge(rating_games, right_on="Username",left_on="right_side",how="left").drop(columns="Username").rename(columns={"BGGId":"right_BGGId"})


In [ ]:
cosine_similarity = rating_games.merge(cosine_similarity, right_on="left_side",left_on="Username",how="right").drop(columns="Username").rename(columns={"BGGId":"left_BGGId"})


In [ ]:
cosine_similarity = cosine_similarity.merge(rating_list, right_on="Username",left_on="right_side",how="left").drop(columns="Username").rename(columns={"Rating":"right_ratings"})


In [ ]:
cosine_similarity = rating_list.merge(cosine_similarity, right_on="left_side",left_on="Username",how="right").drop(columns="Username").rename(columns={"Rating":"left_ratings"})

In [ ]:
def common_Cal(data):
    """Find a list games rated by both users with similar usernames"""
    commonlist = []
    for i,r in data.iterrows():
        commonlist.append(list(set(r.right_BGGId).intersection(set(r.left_BGGId))))
    return commonlist

In [ ]:
cosine_similarity["common_games"] = common_Cal(cosine_similarity)

In [ ]:
cosine_similarity["common_games_count"] = cosine_similarity.common_games.apply(lambda x: len(x))

In [ ]:
cosine_similarity = cosine_similarity.sort_values(by=["similarity","common_games_count"], ascending=False).drop(columns="common_games")

In [ ]:
#cosine_similarity.to_csv("similar usernames.csv")

In [ ]:
cosine_similarity#[cosine_similarity.right_side.str.contains("Adriano Peres")]

#### Another way of finding similar usernames

##### The fuzz package in python can be used for exploring similarity between strings. It uses Levenshtein distance but adjusts for partial similarties like when one string is a subset of another

In [ ]:
# reference https://github.com/thuynh323/Natural-language-processing/blob/master/FuzzyWuzzy%20-%20Ramen%20Rater%20List/fuzz_match_function.py
def get_score(unique_values, scorer, threshold):
    """
    Return a table of each brand name, its corresponding
    similar names found in the list, and their score
    Parameters:
    unique_value - the list of unique values
    scorer - the selected scorer (fuzz.token_sort_ratio, fuzz.token_set_ratio)
    threshold - the cut-off score
    """
    #pd.set_option('display.max_rows', None)
    #pd.set_option('display.max_colwidth', None)
    
    # Create tuples of brand names, matched brand names, and the score
    score = [(x,) + i
             for x in unique_values
             for i in process.extract(x, unique_values, scorer=scorer)]
    
    # Create dataframe from the tuples
    similarity = pd.DataFrame(score, columns=['value','match','score'])
    
    # Derive representative values
    similarity['sorted_value'] = np.minimum(similarity['value'], similarity['match'])
    high_score = similarity[(similarity['score'] >= threshold) &
                            (similarity['value'] != similarity['match']) &
                            (similarity['sorted_value'] != similarity['match'])]
    return high_score
    # Group matches by brand names and scores
#     if scorer == fuzz.token_sort_ratio:
#         result = high_score.groupby(['value','score']).agg({'match': ', '.join}).sort_values(['score'], ascending=False)
#     if scorer == fuzz.token_set_ratio:
#         result = high_score.groupby(['match','score']).agg({'value': ', '.join}).sort_values(['score'], ascending=False)
#     return result


In [ ]:
table = get_score(userRatings["Username"].unique(), fuzz.token_set_ratio, 50)

In [ ]:
table.info()

In [ ]:
table.sort_values(by="score", ascending=False, inplace=True)

In [ ]:
table.drop(columns="sorted_value", inplace=True)

In [ ]:
nameSimilarity = table.reset_index().merge(userRatings.groupby("Username")['BGGId'].apply(list).reset_index(name='match_games'), left_on="match", right_on="Username", how="left").merge(userRatings.groupby("Username")['BGGId'].apply(list).reset_index(name='value_games'), left_on="value", right_on="Username", how="left").drop(columns=["Username_x", "Username_y"])




In [ ]:
nameSimilarity.info()

In [ ]:
nameSimilarity

In [ ]:
def bggidSimilarity(data):
    commonGames = []
    for index, row in tqdm(data.iterrows()):
        commonGames.append(len(list(set(row.match_games).intersection(row.value_games))))
    data["common_games"] = commonGames
    return data

In [ ]:
nameSimilarityTested = bggidSimilarity(nameSimilarity)

In [ ]:
nameSimilarityTested

In [ ]:
nameSimilarityTested[nameSimilarityTested.common_games>0]

In [ ]:
### Next steps
# 1. increase name similarity threshold recursively
# 2. Merge all matches identical to a value: first check if the matches have a game in common
# 3. Replace names dimed to represent the same user

In [ ]:
#gameList = userRatings.groupby("Username")['BGGId'].apply(list)
# for index, row in tqdm(nameSimilarity.iterrows()):
#     if type(row.value_games) != list:
#         if (math.isnan(row.value_games)):
#             valueGames = []
#             for i in row.value.split(", "):
#                 valueGames.extend(gameList[i])
#             nameSimilarity.at[index,'value_games']=valueGames
    

In [ ]:
pivotedTable = userRatings.pivot_table(values="Rating", index="Username",columns="BGGId")

In [ ]:
pivotedTable.isna().sum().sum()/(pivotedTable.shape[0]*pivotedTable.shape[1])

In [ ]:
pivotedTable.isna().sum()

In [ ]:
pivotedTable.shape

In [ ]:
#table = pd.read_csv("user_ratings.csv")

In [ ]:
table.info()